In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from sqlalchemy import create_engine
import sys
import pdfkit
from pdf_mail import sendpdf
import time
import schedule

In [3]:
def recently():
    schedule_datetime=schedule.next_run()
    spotify_client_id = "f0a222565c854f32b07dc713ec215d70"
    spotify_client_secret = "0b853a5a3cb44039835e3b7bef9ad8eb"
    spotify_redirect_url = "http://localhost:8000/redirect/"
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=spotify_client_id,
                                                   client_secret=spotify_client_secret,
                                                   redirect_uri=spotify_redirect_url,
                                                   scope="user-read-recently-played"))
    recently_played = sp.current_user_recently_played(limit=10)

#if the length of recently_played is 0 for some reason just exit the program
    if len(recently_played) ==0:
        sys.exit("No results recieved from Spotify")
    song_list = []
    for row in recently_played['items']:
        song_name = row['track']['name']
        album_release_date = row['track']['album']['release_date']
        artist_name= row['track']['album']['artists'][0]['name']
        song_duration = row['track']['duration_ms']
        song_url = row['track']['external_urls']['spotify']
        song_popularity = row['track']['popularity']
        song_time_played = row['played_at']
        artist_id = row['track']['album']['artists'][0]['id']
        song_element = {'name':song_name,'release_date':album_release_date,'artist_name':artist_name,'duration_ms':song_duration,
                    'url':song_url,'popularity':song_popularity,'date_time_played':song_time_played,'artist_id':artist_id}
        song_list.append(song_element)
    song_df = pd.DataFrame.from_dict(song_list)
    song_df['date_time_played'] = pd.to_datetime(song_df['date_time_played'])
    song_df['date_time_played'] = song_df['date_time_played'].dt.tz_convert('Asia/Kolkata')
    song_df['date_time_played'] = song_df['date_time_played'].astype(str).str[:-7]
    song_df['date_time_played'] = pd.to_datetime(song_df['date_time_played'])
    song_df = song_df[['name','release_date','artist_name','duration_ms','url','popularity','date_time_played','artist_id']]
    song_df.to_csv(r"C:\Users\yuvan shankar\Downloads\spotify\recommendation\recently_10.csv",index=False)

def recommendation():
    schedule_datetime=schedule.next_run()
    spotify_client_id = "f0a222565c854f32b07dc713ec215d70"
    spotify_client_secret = "0b853a5a3cb44039835e3b7bef9ad8eb"
    spotify_redirect_url = "http://localhost:8000/redirect/"
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=spotify_client_id,
                                                   client_secret=spotify_client_secret,
                                                   redirect_uri=spotify_redirect_url))
    df_1= pd.read_csv("recently_10.csv")
    print(df_1)
    qwe=list(df_1.artist_id)
    results = sp.recommendations(seed_artists=[qwe[0]],limit=10)
    if len(results) ==0:
        sys.exit("No results recieved from Spotify")
        
    artist_list = []
    for track in results['tracks']:
        song_name = track['name']
        artist_name= track['album']['artists'][0]['name']
        artist_element = {'song_name':song_name,'artist_name':artist_name}
        artist_list.append(artist_element)
    df = pd.DataFrame.from_dict(artist_list)
    df.to_csv(r"C:\Users\yuvan shankar\Downloads\spotify\recommendation\recommendation_10.csv",index=False)
    csv = 'recommendation_10.csv'
    html_file = csv[:-3]+'html'
    df = pd.read_csv(csv, sep=',')
    df.to_html(html_file)
    path_wkhtmltopdf = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'
    config = pdfkit.configuration(wkhtmltopdf=path_wkhtmltopdf)
    pdfkit.from_url("recommendation_10.html", "FinalOutput.pdf", configuration=config)
    a=sendpdf("sunapana5666@gmail.com",
           "yuvanshankar5666@gmail.com",
           "Sunapana@5666",
           "recommendation songs",
           "list of 10 recommendation songs",
           "FinalOutput",
           "C:/Users/yuvan shankar/Downloads/spotify/recommendation")
    a.email_send()
    print("success")
schedule.every(5).minutes.do(recommendation)
while True:
    schedule.run_pending()
    time.sleep(1)

success
                                                name release_date  \
0                    Doctor Theme - Background Score   2021-10-18   
1                                   Yaar Azhaippadhu   2021-01-01   
2   Two Two Two (From "Kaathuvaakula Rendu Kaadhal")   2021-09-18   
3                     Tum Tum (From "Enemy - Tamil")   2021-09-03   
4                            Cold Heart - PNAU Remix   2021-10-22   
5                  INDUSTRY BABY (feat. Jack Harlow)   2021-09-17   
6  All Too Well (10 Minute Version) (Taylor's Ver...   2021-11-12   
7                                          Oh My God   2021-11-19   
8                          STAY (with Justin Bieber)   2021-07-23   
9                                         Easy On Me   2021-11-19   

           artist_name  duration_ms  \
0  Anirudh Ravichander        43124   
1              Ghibran       250325   
2  Anirudh Ravichander       176345   
3      Various Artists       228567   
4           Elton John       202735  

KeyboardInterrupt: 